In [1]:
import pandas as pd
import ast

* this needs to be integrated into ml utils 2 

In [2]:
# read in zone id with coordinates df
coords_df = pd.read_csv('../Resources/zone_coords_ids.csv')
coords_df['coords'] = coords_df['coords'].apply(ast.literal_eval)
coords_df[['latitude', 'longitude']] = pd.DataFrame(coords_df['coords'].tolist(), index=coords_df.index)
coords_df = coords_df[['LocationID', 'Zone', 'Borough', 'latitude', 'longitude']]
coords_df.head()

,LocationID,Zone,Borough,latitude,longitude
0,1,Newark Airport,EWR,40.689064,-74.177255
1,2,Jamaica Bay,Queens,40.603994,-73.835412
2,3,Allerton/Pelham Gardens,Bronx,40.865430,-73.867365
3,4,Alphabet City,Manhattan,40.725102,-73.979583
4,5,Arden Heights,Staten Island,40.563700,-74.191603


In [3]:
# read in model_df with PU and DO Ids
results = pd.read_parquet('Resources/model_df_2.parquet')

results.columns = results.columns.str.replace('(', '').str.replace("'", "").str.replace(',', '').str.replace(')', '')

results.head()

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,PULocationID,DOLocationID,prcp,temp,distance,durationsec,fare,tolls,airport,congestion,total,class
0,5,1,False,1,0,0,132,41,0.0,42.8,18.85,1599.0,70.00,6.94,1.75,0.0,95.88,0
1,9,1,False,1,0,0,3,51,0.0,42.8,2.47,785.0,14.59,0.00,0.00,0.0,14.59,1
2,33,1,False,1,0,0,145,112,0.0,42.8,2.02,474.0,20.62,0.00,0.00,0.0,20.62,2
3,38,1,False,1,0,0,16,73,0.0,42.8,2.47,459.0,23.20,0.00,0.00,0.0,23.20,2
4,42,1,False,1,0,0,164,161,0.0,42.8,1.06,335.0,7.90,0.00,0.00,2.5,15.48,0


In [4]:
# define pickup and drop off coords and merge with results
pickup_coords = coords_df[['LocationID', 'latitude', 'longitude']].copy()
pickup_coords.rename(columns={'LocationID': 'PULocationID',
                                'latitude': 'PUy',  
                                'longitude': 'PUx'},  
                      inplace=True)

results_with_pu = pd.merge(results, pickup_coords, on='PULocationID', how='left')


dropoff_coords = coords_df[['LocationID', 'latitude', 'longitude']].copy()
dropoff_coords.rename(columns={'LocationID': 'DOLocationID',
                                'latitude': 'DOy',
                                'longitude': 'DOx'},
                       inplace=True)

results_with_coords = pd.merge(results_with_pu, dropoff_coords, on='DOLocationID', how='left')


results_with_coords.head()

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,PULocationID,DOLocationID,prcp,temp,...,fare,tolls,airport,congestion,total,class,PUy,PUx,DOy,DOx
0,5,1,False,1,0,0,132,41,0.0,42.8,...,70.00,6.94,1.75,0.0,95.88,0,40.642948,-73.779373,40.808900,-73.948200
1,9,1,False,1,0,0,3,51,0.0,42.8,...,14.59,0.00,0.00,0.0,14.59,1,40.865430,-73.867365,40.873889,-73.829444
2,33,1,False,1,0,0,145,112,0.0,42.8,...,20.62,0.00,0.00,0.0,20.62,2,40.741509,-73.956975,40.723713,-73.950971
3,38,1,False,1,0,0,16,73,0.0,42.8,...,23.20,0.00,0.00,0.0,23.20,2,40.763120,-73.770745,40.757537,-73.784305
4,42,1,False,1,0,0,164,161,0.0,42.8,...,7.90,0.00,0.00,2.5,15.48,0,40.749842,-73.984251,40.765064,-73.985319


In [5]:
results_with_coords.columns

Index(['second_of_day', 'day_of_year', 'weekend', 'holiday', 'morning_rush',
       'evening rush', 'PULocationID', 'DOLocationID', 'prcp', 'temp',
       'distance', 'durationsec', 'fare', 'tolls', 'airport', 'congestion',
       'total', 'class', 'PUy', 'PUx', 'DOy', 'DOx'],
      dtype='object')

In [6]:
# reorganize columns
results_with_coords = results_with_coords [['second_of_day', 'day_of_year', 'weekend', 'holiday', 'morning_rush',
       'evening rush', 'PULocationID', 'PUy', 'PUx', 'DOLocationID', 'DOy', 'DOx', 'prcp', 'temp',
       'distance', 'durationsec', 'fare', 'tolls', 'airport', 'congestion',
       'total', 'class']]

results_with_coords

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,PULocationID,PUy,PUx,DOLocationID,...,prcp,temp,distance,durationsec,fare,tolls,airport,congestion,total,class
0,5,1,False,1,0,0,132,40.642948,-73.779373,41,...,0.0,42.80,18.85,1599.0,70.00,6.94,1.75,0.0,95.88,0
1,9,1,False,1,0,0,3,40.865430,-73.867365,51,...,0.0,42.80,2.47,785.0,14.59,0.00,0.00,0.0,14.59,1
2,33,1,False,1,0,0,145,40.741509,-73.956975,112,...,0.0,42.80,2.02,474.0,20.62,0.00,0.00,0.0,20.62,2
3,38,1,False,1,0,0,16,40.763120,-73.770745,73,...,0.0,42.80,2.47,459.0,23.20,0.00,0.00,0.0,23.20,2
4,42,1,False,1,0,0,164,40.749842,-73.984251,161,...,0.0,42.80,1.06,335.0,7.90,0.00,0.00,2.5,15.48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5971769,85846,366,False,0,0,0,58,40.839167,-73.819722,213,...,0.0,48.02,2.48,534.0,14.38,0.00,0.00,0.0,14.38,2
5971770,85864,366,False,0,0,0,189,40.677871,-73.968473,62,...,0.0,48.02,1.58,488.0,15.01,0.00,0.00,0.0,15.01,2
5971771,85870,366,False,0,0,0,79,40.729269,-73.987361,79,...,0.0,48.02,0.60,256.0,5.80,0.00,0.00,2.5,10.80,0
5971772,86042,366,False,0,0,0,180,40.676770,-73.843746,63,...,0.0,48.02,1.36,266.0,8.92,0.00,0.00,0.0,8.92,1


In [8]:
for column in results_with_coords:
    print(results_with_coords[column].value_counts())

second_of_day
27000    304
36000    299
28800    291
32400    285
30600    267
        ... 
15399      6
16068      6
12903      5
15191      5
18417      5
Name: count, Length: 86400, dtype: int64
day_of_year
69     37833
62     36098
83     35069
61     32137
68     32072
       ...  
126     4974
141     4970
146     4259
147     4045
148     3872
Name: count, Length: 366, dtype: int64
weekend
False    4161122
True     1810652
Name: count, dtype: int64
holiday
0    5833924
1     137850
Name: count, dtype: int64
morning_rush
0    5346033
1     625741
Name: count, dtype: int64
evening rush
0    5212234
1     759540
Name: count, dtype: int64
PULocationID
132    156307
138    141016
161    137764
237    126886
236    113690
        ...  
99         73
2          12
110         8
199         2
105         1
Name: count, Length: 261, dtype: int64
PUy
40.768311    422362
40.642948    156307
40.775714    141016
40.765064    137764
40.766437    136014
              ...  
40.577744        73


In [9]:
# save results with pu and do coords
results_with_coords.to_parquet('../Resources/model_coords_df.parquet')